In [52]:
import pandas as pd
import requests

## Creating Request to Search for Physicians on Yelp.com
You will require an API Key to extract data, used two functions: request() and getreviews()

To create an API Key, do the following:
- sign up on Yelp.com to create an account, ignore if already have an account
- Login to Yelp.com
- Go to Create App
- In the create new app form, enter dummy information about your app(App Name, Industry, Email, Description), then agree to Yelp API Terms of Use and Display Requirements. Then click the Submit button.
- You will now have an API Key.
- Replace the Fake API Key in both functions with your API Key

In [41]:
def request():
    """Given your API_KEY, send a GET request to the API.
    Args:
        url (str): The path of the API with the domain.
        api_key (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    api_key = "ajndlajsnashjksdbjknsdghjklljfghjkldfghjklgfdghjkhgfdghjkhgfdhhg" # Fake API Key
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }
    url_params = {
        'term': "physician".replace(' ', '+'),
        'location': "New York, NY".replace(' ', '+'),
        'limit': '50'
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()

In [43]:
yelp_data = request()

Querying https://api.yelp.com/v3/businesses/search ...


### Get Reviews for Each Physician using Yelp Fusion API

In [68]:
def get_reviews(business_id):
    """Given your API_KEY, send a GET request to the API.
    Args:
        url (str): The path of the API with the domain.
        api_key (str): Your API Key.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url = "https://api.yelp.com/v3/businesses/{buss_id}/reviews".format(buss_id=business_id)
    api_key = "ajndlajsnashjksdbjknsdghjklljfghjkldfghjklgfdghjkhgfdghjkhgfdhhg" # Fake API Key

    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }
    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers)
    return response.json()

### Filtering Required Data into Pandas DataFrame

In [143]:
yelp_dataframe = pd.DataFrame()
for data in yelp_data["businesses"]:
    avg_rating = data["rating"]
    title = data["categories"][0]["title"]
    alias = data["categories"][0]["alias"]
    city = data["location"]["city"]
    country = data["location"]["country"]
    address = ", ".join(data["location"]["display_address"])
    state = data["location"]["state"]
    zip_code = data["location"]["zip_code"]
    name = data["name"]
    buss_id = data["id"]
    response_dict = get_reviews(buss_id)
    if response_dict:
        reviews_list = response_dict["reviews"]
        for review in reviews_list:
            review["user"] = review["user"]["name"]
            review["physician"] = name
            review["avg_rating"] = avg_rating
            review["title"] = title
            review["alias"] = alias
            review["city"] = city
            review["country"] = country
            review["address"] = address
            review["state"] = state
            review["zip_code"] = zip_code
        yelp_dataframe = yelp_dataframe.append(reviews_list)

Querying https://api.yelp.com/v3/businesses/laura-guderian-md-new-york/reviews ...
Querying https://api.yelp.com/v3/businesses/mohawk-village-medical-new-york-2/reviews ...
Querying https://api.yelp.com/v3/businesses/michael-raffinan-md-new-york/reviews ...
Querying https://api.yelp.com/v3/businesses/inna-grishina-md-brooklyn-2/reviews ...
Querying https://api.yelp.com/v3/businesses/xiaoyan-song-md-flushing/reviews ...
Querying https://api.yelp.com/v3/businesses/bruce-chung-md-new-york/reviews ...
Querying https://api.yelp.com/v3/businesses/weill-cornell-medicine-primary-care-west-side-new-york-3/reviews ...
Querying https://api.yelp.com/v3/businesses/socrates-t-kangadis-md-new-york/reviews ...
Querying https://api.yelp.com/v3/businesses/karis-cho-md-new-york/reviews ...
Querying https://api.yelp.com/v3/businesses/jason-h-dolinsky-md-new-york-2/reviews ...
Querying https://api.yelp.com/v3/businesses/michelle-li-md-new-york/reviews ...
Querying https://api.yelp.com/v3/businesses/aldo-al

In [144]:
yelp_dataframe.reset_index(inplace=True)

In [145]:
yelp_dataframe.drop('index', axis=1, inplace=True)

In [148]:
yelp_dataframe.head(2)

,address,alias,avg_rating,city,country,physician,rating,state,text,time_created,title,url,user,zip_code
0,"35 E 21st St, Fl 7, New York, NY 10010",internalmed,4.5,New York,US,"Laura Guderian, MD",5,NY,I've been delighted with Dr. Guderian as my pr...,2017-09-06 13:21:59,Internal Medicine,https://www.yelp.com/biz/laura-guderian-md-new...,Bonnie E.,10010
1,"35 E 21st St, Fl 7, New York, NY 10010",internalmed,4.5,New York,US,"Laura Guderian, MD",1,NY,I really wanted to like One Medical Group. I a...,2016-09-25 16:00:50,Internal Medicine,https://www.yelp.com/biz/laura-guderian-md-new...,Yelpish N.,10010


### Exporting Pandas DataFrame to Local CSV

In [149]:
yelp_dataframe.to_csv("yelp_reviews.csv", index=False, encoding="UTF-8")